### Matchup Winrate

This program will:

1) Sum up the top winrates of Masters+ in EUW and KR Solo Queue

2) Of these top win rates, we will figure out the loss rate versus champions

### Prompt 1

In [1]:
import requests
import json

In [18]:
# Asks for an updated API Key
APIKey = (str)(input('Copy and paste your API Key here: '))

Copy and paste your API Key here: RGAPI-e146ba91-f717-4b5b-8c61-401cf9134632


In [12]:
# Puts the challenger information into a json file
URL = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + APIKey
chal_resp = requests.get(URL)
json_chal_resp = chal_resp.json()

# Print out what the JSON file looks like
# print(json.dumps(json_chal_resp, indent = 3, sort_keys=True))

# Initializes a list to store the encrypted summoner IDs
summIds = []

# Places all the challenger player summmoner IDs into the list
for i in range(len(json_chal_resp['entries'])):
    # there are always 300 players in challenger queues
    summIds.append(json_chal_resp['entries'][0]['summonerId'])

# Repeat the process done for the challenger players with master players
URL2 = 'https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + APIKey
mast_resp = requests.get(URL2)
json_mast_resp = mast_resp.json()
for i in range(len(json_mast_resp['entries'])):
    summIds.append(json_mast_resp['entries'][0]['summonerId'])

Copy and paste your API Key here: RGAPI-d00f3d03-0e8b-4758-9310-2be5269be990


In [ ]:
# We currently have encrypted summonerID
# To get a matchlist, we need the encryptedAccountID

In [14]:
# Testing getting the accountID through summonerID
accId = []
for i in range(len(summIds)):
    acc = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + summIds[i] + '?api_key=' + APIKey
    acc_resp = requests.get(acc)
    json_acc_resp = acc_resp.json()
    accId.append(json_acc_resp['accountId'])
    # The current problem is that we can only execute this so many times before our API key rate limit is exceeded

KeyError: 'accountId'

In [20]:
acc = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + summIds[2] + '?api_key=' + APIKey
acc_resp = requests.get(acc)
json_acc_resp = acc_resp.json()
print(json.dumps(json_acc_resp, indent = 3, sort_keys=True))


{
   "accountId": "PuTzcGbyO8X0kk9E7-ezJJs1YC5Tv8dZKBtNQ_qmIvMfqvtA3diOTKcb",
   "id": "VFZtlGvKGgCSIgdP2ZtOLTGLk0_16ICoTT2zTOFJiWDFHWCJ",
   "name": "FearTheSticks",
   "profileIconId": 4378,
   "puuid": "fEnqS081PvajNALxAQoI3wiwJa7x1oVgSyHmnwGfGIWGWJ54OGSOaAueB1wGdruDIREWVsErkfxuNg",
   "revisionDate": 1573428171000,
   "summonerLevel": 115
}


In [22]:
# We can get a certain amount of data at a time and then database it into a table
accId = 'PuTzcGbyO8X0kk9E7-ezJJs1YC5Tv8dZKBtNQ_qmIvMfqvtA3diOTKcb'
URL3 = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accId + '?api_key=' + APIKey
json_match_resp = requests.get(URL3).json()
print(json.dumps(json_match_resp, indent = 3, sort_keys=True))

{
   "endIndex": 100,
   "matches": [
      {
         "champion": 20,
         "gameId": 3202215815,
         "lane": "JUNGLE",
         "platformId": "NA1",
         "queue": 420,
         "role": "NONE",
         "season": 13,
         "timestamp": 1573426385598
      },
      {
         "champion": 20,
         "gameId": 3202209510,
         "lane": "JUNGLE",
         "platformId": "NA1",
         "queue": 420,
         "role": "NONE",
         "season": 13,
         "timestamp": 1573424691335
      },
      {
         "champion": 9,
         "gameId": 3202003737,
         "lane": "BOTTOM",
         "platformId": "NA1",
         "queue": 420,
         "role": "DUO_SUPPORT",
         "season": 13,
         "timestamp": 1573409911152
      },
      {
         "champion": 9,
         "gameId": 3201969366,
         "lane": "BOTTOM",
         "platformId": "NA1",
         "queue": 420,
         "role": "DUO_SUPPORT",
         "season": 13,
         "timestamp": 1573407920741
      },
  

In [23]:
# Returns the given matchId
json_match_resp['matches'][1]['gameId']

3202209510

In [28]:
# Getting information about the match
match = 'https://na1.api.riotgames.com/lol/match/v4/matches/' + str(json_match_resp['matches'][1]['gameId']) + '?api_key=' + APIKey
match_info = requests.get(match).json()
print(json.dumps(match_info, indent = 2, sort_keys=True))
# We want to find what champion won in addition to who they were against
# We need to keep a sum of all of these lane matchups or keep track of who won
# We can make a pairing of all key value pairing with certain matchups for every matchup and then iterating that value 
# every time the matchup is won by that person
# From there, we can then create a graph depicting highest win rate overall and sorting it by the specified champion


{
  "gameCreation": 1573424691335,
  "gameDuration": 1365,
  "gameId": 3202209510,
  "gameMode": "CLASSIC",
  "gameType": "MATCHED_GAME",
  "gameVersion": "9.22.296.5720",
  "mapId": 11,
  "participantIdentities": [
    {
      "participantId": 1,
      "player": {
        "accountId": "YgVD-9X9CafzBiGWyO2etgoWRo7WaR55wuxsHGvS0hcSEC0",
        "currentAccountId": "YgVD-9X9CafzBiGWyO2etgoWRo7WaR55wuxsHGvS0hcSEC0",
        "currentPlatformId": "NA1",
        "matchHistoryUri": "/v1/stats/player_history/NA1/239157653",
        "platformId": "NA1",
        "profileIcon": 7,
        "summonerId": "OYoRyAWMk382cquqyAr7csXJokjZTzpA9YeDeHKme7bl3do",
        "summonerName": "AcilioN"
      }
    },
    {
      "participantId": 2,
      "player": {
        "accountId": "mVwRFLdI9eRuCMeKVYfRD74jwyfM6JSDeVPGhUufyjjuq6k",
        "currentAccountId": "mVwRFLdI9eRuCMeKVYfRD74jwyfM6JSDeVPGhUufyjjuq6k",
        "currentPlatformId": "NA1",
        "matchHistoryUri": "/v1/stats/player_history/NA1/2233703